# Dataset Exploration Notebook

## Introduction
This notebook demonstrates the process of securely accessing a dataset stored in Google Sheets using the Google Cloud Platform (GCP) Service Account. This initial section outlines key milestones achieved to set up a secure and efficient workflow for this project.

## Milestones Achieved
1. **GCP Service Account Configuration**:
   - Successfully created and configured a GCP Service Account to access the Google Sheets API.
   - Shared the dataset file in Google Sheets with the service account email to grant edit permissions.

2. **Secure Credential Management**:
   - Implemented the use of environment variables to securely store and access the path to the service account JSON file.
   - Ensured the JSON credentials file is not committed to the repository by:
     - Adding the `secrets/` directory and `.env` file to the `.gitignore`.
     - Setting up environment variables dynamically during virtual environment activation.

3. **Virtual Environment Setup**:
   - Created and utilized a Python virtual environment for package management.
   - Ensured the virtual environment includes all required dependencies, isolating the project environment from the global system.

4. **Data Access**:
   - Successfully retrieved the dataset from Google Sheets into a Pandas DataFrame for analysis, confirming seamless integration between GCP and the project.



---


The code below does the following:
- Imports the necessary libraries: 

    -`os` to extract environment variables.

    -`Credentials` class from the `google.oauth2.service_account` module for authenticating the service account.

In [8]:
import os
from google.oauth2.service_account import Credentials

# Get the path from the environment variable
json_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")


The 'GOOGLE_APPLICATION_CREDENTIALS' environment variable is set to the path of the service account JSON file. This variable is used by the Google Cloud client libraries to locate the service account credentials.

Now, as the next step we can import the following libraries:

- `gspread` to interact with Google Sheets.
- `pandas` to work with the dataset.


Also, some classes, such as `Credentials` from the `google.oauth2.service_account` module, and `Request` from the `google.auth.transport.requests` module are imported to authenticate the service account and make requests to the Google Sheets API.



In [9]:
import gspread
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
import pandas as pd

Now, we will get the environment variable that holds the path to the gcp service account .json file, which is not commited to this repository (included in the .gitignore file). If you wish to do the same I recommend to edit the activate script of your virtual environment to set the environment variable every time you activate it. 

the `getenv` function from the `os` module is used to get the value of the 'GOOGLE_APPLICATION_CREDENTIALS' environment variable and saves it into the `json_path` variable.

In [10]:
json_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
if not json_path:
    raise FileNotFoundError("Environment variable GOOGLE_APPLICATION_CREDENTIALS is not set or file path is invalid.")

then, the `json_path` variable is used to authenticate the service account and access the Google Sheets API. The `Credentials.from_service_account_file` method is used to create credentials from the service account JSON file.

In [11]:
# Step 2: Authenticate using the service account JSON file
scopes = ["https://www.googleapis.com/auth/spreadsheets.readonly"]
credentials = Credentials.from_service_account_file(json_path, scopes=scopes)

now the `gspread.authorize` method is used to authenticate the service account and access the Google Sheets API. 

this information is saved in the `client` variable, which is used to access the Google Sheets API.

In [12]:
# Step 3: Authorize the gspread client
client = gspread.authorize(credentials)

now, we can pass the spreadsheet url to the `open_by_url` method of the `client` object to access the Google Sheets file. 

In [13]:
# Step 4: Open the Google Sheet by URL
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1f7jIcEzhLiO2EhVZro8oUdNgm2AGaC5po8QNiuQggG4/edit?usp=sharing"
spreadsheet = client.open_by_url(spreadsheet_url)

Now we select the first worksheet of the Google Sheets file using the `get_worksheet` method of the `spreadsheet` object.

In [ ]:
# Step 5: Select the worksheet (e.g., first worksheet)
worksheet = spreadsheet.get_worksheet(0)

Once the worksheet is selected, we can fetch all the records from the worksheets as a list of dictionaries. This is done using the `get_all_records` method of the `worksheet` object.

Then, we can convert the list of dictionaries to a Pandas DataFrame using the `pd.DataFrame` constructor. This will allow us to perform data analysis and visualization on the dataset.

Finally, we can display the first few rows of the dataset using the `head` method of the Pandas DataFrame.  


In [15]:
# Step 6: Load data into a pandas DataFrame
data = worksheet.get_all_records()
df = pd.DataFrame(data)

# Display the first few rows
print(df.head())


   # claim_status    video_id  video_duration_sec  \
0  1        claim  7017666017                  59   
1  2        claim  4014381136                  32   
2  3        claim  9859838091                  31   
3  4        claim  1866847991                  25   
4  5        claim  7105231098                  19   

                            video_transcription_text verified_status  \
0  someone shared with me that drone deliveries a...    not verified   
1  someone shared with me that there are more mic...    not verified   
2  someone shared with me that american industria...    not verified   
3  someone shared with me that the metro of st. p...    not verified   
4  someone shared with me that the number of busi...    not verified   

  author_ban_status video_view_count video_like_count video_share_count  \
0      under review           343296            19425               241   
1            active           140877            77355             19034   
2            active    

Now that we have access to the data, we can proceed to do some Exploratory Data Analysis.

# Exploratory Data Analysis

The particular interest of this project is explore information related to what disntinguishes claim videos from opinion videos.

Now we will assess the data. we already checked the dataset contents using the .head() method, now we will check the data types of the columns using the .info() method.

In [16]:
df.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19382 entries, 0 to 19381
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   #                         19382 non-null  int64 
 1   claim_status              19382 non-null  object
 2   video_id                  19382 non-null  int64 
 3   video_duration_sec        19382 non-null  int64 
 4   video_transcription_text  19382 non-null  object
 5   verified_status           19382 non-null  object
 6   author_ban_status         19382 non-null  object
 7   video_view_count          19382 non-null  object
 8   video_like_count          19382 non-null  object
 9   video_share_count         19382 non-null  object
 10  video_download_count      19382 non-null  object
 11  video_comment_count       19382 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.8+ MB


Descriptive statistics can also be used in this stage of EDA to summarize the central tendency, dispersion, and shape of the dataset’s distribution. This can be done using the .describe() method.

In [17]:
df.describe()

,#,video_id,video_duration_sec
count,19382.000000,1.938200e+04,19382.000000
mean,9691.500000,5.627454e+09,32.421732
std,5595.245794,2.536440e+09,16.229967
min,1.000000,1.234959e+09,5.000000
25%,4846.250000,3.430417e+09,18.000000
50%,9691.500000,5.618664e+09,32.000000
75%,14536.750000,7.843960e+09,47.000000
max,19382.000000,9.999873e+09,60.000000


Now to assess the distributions of the variables and identify outliers, we can create box plots for the numerical columns and histograms for the categorical columns.